# Notebook Processo Seletivo Aluno Especial IA-024 1S2024 FEEC-UNICAMP
versão 5 de fevereiro de 2024, 19h

In [1]:
!pip install torchtext
!pip install 'portalocker>=2.0.0'

ERROR: Invalid requirement: "'portalocker"


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.datasets import IMDB
from collections import Counter
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import get_tokenizer

## I - Vocabulário e Tokenização

In [3]:
# limit the vocabulary size to 20000 most frequent tokens
vocab_size = 20000

#n_samples = 200
n_samples = 2500

# I.1. Na célula de calcular o vocabulário, aproveite o laço sobre IMDB de treinamento e utilize um segundo contador
# para calcular o número de amostras positivas e amostras negativas.
# Calcule também o comprimento médio do texto em número de palavras dos textos das amostras.

counter = Counter()
counter_lbl = Counter({"pos": 0, "neg": 0, "total": 0})
total_review_len = 0
avg_review_len = 0

for (label, line) in list(IMDB(split='train'))[:n_samples]:
    counter.update(line.split())

    # Número de amostras positivas e negativas
    if (label == 1):
      counter_lbl['neg'] += 1
    else:
      counter_lbl['pos'] += 1
    counter_lbl['total'] += 1

    # Comprimento médio do texto das reviews em palavras
    tokenizer = get_tokenizer('basic_english')

    # tokenize the sentence
    tokens = tokenizer(line)

    # count the number of words
    total_review_len += len(tokens)

# Comprimento médio
avg_review_len = total_review_len / counter_lbl['total']

# I.2 Mostre as cinco palavras mais frequentes do vocabulário e as cinco palavras menos frequentes.
# Qual é o código do token que está sendo utilizado quando a palavra não está no vocabulário?
# Calcule quantos tokens das frases do conjunto de treinamento que não estão no vocabulário.

# create a vocabulary of the 20000 most frequent tokens
most_frequent_words = sorted(counter, key=counter.get, reverse=True)[:vocab_size]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)} # words indexed from 1 to 20000
vocab_size = len(vocab) #Errata

print("Amostras positivas, negativas e totais:")
print(counter_lbl)
print()

print("Comprimento médio do texto em palavras")
print(avg_review_len)
print()

print("Cinco palavras mais frequentes:")
print(most_frequent_words[:5])
print()

print("Cinco palavras menos frequentes:")
print(most_frequent_words[-5:])
print()

Amostras positivas, negativas e totais:
Counter({'neg': 2500, 'total': 2500, 'pos': 0})

Comprimento médio do texto em palavras
263.7684

Cinco palavras mais frequentes:
['the', 'a', 'and', 'of', 'to']

Cinco palavras menos frequentes:
['concealed', 'Estella', 'Daena', 'plans,', 'technological']



In [4]:
# I.2 Calcule quantos tokens das frases do conjunto de treinamento que não estão no vocabulário.

def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in sentence.split()] # 0 for OOV

encode_sentence("I like Pizza", vocab)

# Cálculo do número de tokens que não estão no vocabulário na base de treinamento:
tokens = []
for (label, line) in list(IMDB(split='train'))[:n_samples]:
  tokens.extend(encode_sentence(line, vocab))

print("Número de tokens que não estão no vocabulário na base de treinamento:")
print(tokens.count(0))

Número de tokens que não estão no vocabulário na base de treinamento:
43515


#### I.2 Qual é o código do token que está sendo utilizado quando a palavra não está no vocabulário?

Na função de dicionário dict.get() o segundo parâmetro indica o valor default caso a palavra não seja encontrada no dicionário. Nesse caso o código do token usado é o número zero.


#### I.3.a) Qual é a razão pela qual o modelo preditivo conseguiu acertar 100% das amostras de teste do dataset selecionado com apenas as primeiras 200 amostras?

Ao reduzirmos a base de treinamento para apenas 200 amostras, a base se tornou totalmente desbalanceada. Como pudemos verificar, temos 200 amostras classificadas como negativas e nenhuma como positiva.
Portanto a taxa de acurácia calculada sobre a classificação da base de testes depende unicamente da percentagem de amostras positivas ou negativas nesta base.

#### I.3.b) Modifique a forma de selecionar 200 amostras do dataset, porém garantindo que ele continue balanceado, isto é, aproximadamente 100 amostras positivas e 100 amostras negativas.

Para obtermos um dataset balanceado, usaremos uma função que seleciona amostras do dataset de acordo com a classificação e cria um dataset com a quantidade de amostras de cada classificação desejada conforme abaixo.

In [5]:
# Função para selecionar dados balanceados

def balanced_dataset(data, size):
  data_pos = [(label,line) for label, line in data if label == 2][:int(size/2)]
  data_neg = [(label,line) for label, line in data if label == 1][:int(size/2)]

  data_bal = data_pos + data_neg

  return data_bal

# Aplicando sobre a base de treinamento

train_data = IMDB(split='train')
counter = Counter()
total_review_len = 0
avg_review_len = 0

for (label, line) in list(balanced_dataset(train_data, n_samples)):
    counter.update(line.split())

    # Comprimento médio do texto das reviews em palavras
    tokenizer = get_tokenizer('basic_english')

    # tokenize the sentence
    tokens = tokenizer(line)

    # count the number of words
    total_review_len += len(tokens)

# Comprimento médio
avg_review_len = total_review_len / n_samples

print("Comprimento médio do texto em palavras na base balanceada")
print(avg_review_len)
print()

# create a vocabulary of the 20000 most frequent tokens
#most_frequent_words = sorted(counter, key=counter.get, reverse=True)[:vocab_size]
#vocab = {word: i for i, word in enumerate(most_frequent_words, 1)} # words indexed from 1 to 20000
#vocab_size = len(vocab)

Comprimento médio do texto em palavras na base balanceada
267.2208



## II - Dataset

In [6]:
from torch.nn.functional import one_hot
# Dataset Class with One-hot Encoding
class IMDBDataset(Dataset):
    def __init__(self, split, vocab):
        #self.data = list(IMDB(split=split))[:n_samples]
        self.data = list(balanced_dataset(IMDB(split=split), n_samples))        
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, line = self.data[idx]
        label = 1 if label == 1 else 0

        # one-hot encoding
        X = torch.zeros(len(self.vocab) + 1)
        for word in encode_sentence(line, self.vocab):
            X[word] = 1

        return X, torch.tensor(label)

# Load Data with One-hot Encoding
train_data = IMDBDataset('train', vocab)
test_data = IMDBDataset('test', vocab)

# II.1.a) Investigue o dataset criado na linha 24. Faça um código que aplique um laço sobre o dataset train_data
# e calcule novamente quantas amostras positivas e negativas do dataset.
# II.1.b) Calcule também o número médio de palavras codificadas em cada vetor one-hot.
# Compare este valor com o comprimento médio de cada texto (contado em palavras), conforme calculado no exercício

counter_lbl = Counter({"pos": 0, "neg": 0, "total": 0})
words_encoded = 0
for (oneHot, sentiment) in train_data:

    words = oneHot.tolist()
    label = sentiment.item()

    # Número de amostras positivas e negativas
    if (label == 1):
      counter_lbl['neg'] += 1
    else:
      counter_lbl['pos'] += 1
    counter_lbl['total'] += 1

    hot_encoded = sum(words[i] for i in range(len(words)) if words[i] != 0)
    words_encoded +=  hot_encoded

avg_words_enc = words_encoded / counter_lbl['total']

print("Amostras positivas, negativas e totais:")
print(counter_lbl)
print()

print("Quantidade média de palavras codificadas em cada vetor one-hot")
print(avg_words_enc)
print()

Amostras positivas, negativas e totais:
Counter({'total': 2500, 'pos': 1250, 'neg': 1250})

Quantidade média de palavras codificadas em cada vetor one-hot
131.6824



#### II.1.b Compare este valor com o comprimento médio de cada texto (contado em palavras), conforme calculado no exercício I.1.c. e explique a diferença.

No exercício I.1.c, o comprimento médio do texto em palavras depois de passar pelo tokenizador foi de cerca de 265 palavras. Essa diferença do vetor One-Hot se deve ao fato que o vetor one-hot só codifica as palavras que foram identificadas no dicionário, enquanto que o comprimento médio considera todas as palavras das sentenças. Ou seja, palavras que não foram codificadas no dicionário serão representadas por zeros.

## III - Data Loader

In [7]:
#batch_size = 128
batch_size = 256
# define dataloaders

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, pin_memory=True)

## IV - Modelo

In [8]:
class OneHotMLP(nn.Module):
    def __init__(self, vocab_size):
        super(OneHotMLP, self).__init__()
        self.fc1 = nn.Linear(vocab_size+1, 200)
        self.fc2 = nn.Linear(200, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        o = self.fc1(x.float())
        o = self.relu(o)
        return self.fc2(o)

# Model instantiation
model = OneHotMLP(vocab_size)

## V - Laço de Treinamento - Otimização da função de Perda pelo Gradiente descendente

In [9]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print('GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('using CPU')

GPU: NVIDIA GeForce RTX 2060


In [10]:
# II.2 Com a o notebook configurado para GPU T4, meça o tempo de dois laços dentro do
# for da linha 13 (coloque um break após dois laços) e determine quanto demora
# demora para o passo de forward (linhas 14 a 18), para o backward (linhas 20, 21 e 22)
# e o tempo total de um laço. Faça as contas e identifique o trecho que é mais demorado.
# II.2.a) Tempo do laço = ; Tempo do forward = ;Tempo do backward = ; Conclusão.

import time

# Debug
print_loop = False

model = model.to(device)
# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()

# II.2.b) Trecho que precisa ser otimizado. (Esse é um problema mais difícil)
# II.2.c) Otimize o código e explique aqui.

optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    start_time = time.time()  # Start time of the epoch
    model.train()

    loop_count = 0

    for inputs, labels in train_loader:
        loop_start = time.time()
        if(loop_count == 2 and print_loop):
          # Para medição do tempo do loop.
          break

        forward_start = time.time()
        inputs = inputs.to(device)
        labels = labels.to(device)
        gpu_cpy_time = time.time() - forward_start
        # Forward pass
        model_start = time.time()
        outputs = model(inputs)
        model_time = time.time() - model_start
        loss = criterion(outputs.squeeze(), labels.float())
        forward_time = time.time() - forward_start

        # Backward and optimize
        backward_start = time.time()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        backward_time = time.time() - backward_start

        # Loop optimization
        loop_count += 1
        loop_time = time.time() - loop_start
        if (epoch == 0 and print_loop):
          print("Loop #", loop_count)
          print("Tempo de loop = ", loop_time)
          print("Forward pass = ", forward_time)
          print("Gpu cpy =", gpu_cpy_time)
          print("Model =", model_time)
          print("Backward pass = ", backward_time)
          print()

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch

    print(f'Epoch [{epoch+1}/{num_epochs}], \
            Loss: {loss.item():.4f}, \
            Elapsed Time: {epoch_duration:.2f} sec')

Epoch [1/5],             Loss: 0.6927,             Elapsed Time: 9.30 sec
Epoch [2/5],             Loss: 0.6934,             Elapsed Time: 5.89 sec
Epoch [3/5],             Loss: 0.6927,             Elapsed Time: 5.76 sec
Epoch [4/5],             Loss: 0.6930,             Elapsed Time: 5.84 sec
Epoch [5/5],             Loss: 0.6930,             Elapsed Time: 5.79 sec


#### II.2.a) Medição dos tempos de loop

Notamos que o tempo do passo do forward leva mais tempo que o passo de backward, conforme os dados obtidos abaixo para a primeira época do treinamento.

**Para 200 amostras**:

```
Loop # 1
Tempo de loop =  0.7379188537597656
Forward pass =  0.34381794929504395
Backward pass =  0.09396243095397949

Loop # 2
Tempo de loop =  0.9603068828582764
Forward pass =  0.0016582012176513672
Backward pass =  0.004204511642456055
```
#### II.2.b) Trecho que precisa ser otimizado. (Esse é um problema mais difícil)
#### II.2.c) Otimize o código e explique aqui.

Para otimizarmos o loop, podemos utilizar um otimizador mais eficiente, como é o caso do Adam. Alteramos este trecho no código:

```
#optimizer = optim.SGD(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)
```

Notamos os novos resultados para **200 amostras**:

```
Loop # 1
Tempo de loop =  0.27255821228027344
Forward pass =  0.002206563949584961
Backward pass =  0.0019736289978027344

Loop # 2
Tempo de loop =  0.4292025566101074
Forward pass =  0.0016396045684814453
Backward pass =  0.0016634464263916016
```

#### Novos tempos de 

## VI - Avaliação

In [11]:
## evaluation
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        predicted = torch.round(torch.sigmoid(outputs.squeeze()))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')

Test Accuracy: 50.04%
